In [1]:
import requests
import lxml.html
import pandas as pd
from pandas.io import sql
import os
import time
import datetime

import re 
import string 
import json
import numpy as np
from tqdm import tqdm

from selenium import webdriver
from bs4 import BeautifulSoup

In [10]:
import warnings

warnings.filterwarnings(action='ignore') #warning 메시지 안보이게

In [11]:
driver=webdriver.Chrome('chromedriver.exe')

#구독 많은 순
# office_data=[['중앙일보','025'],['JTBC','437'],['YTN','052'],['한국경제','015'],['매일경제','009'],
#             ['조선일보','023'],['연합뉴스','422'],['SBS','055'],['KBS','056'],['한겨레','028']]

#경제지
# office_data=[['매일경제','009'],['머니투데이','008'],['비즈니스워치','648'],['서울경제','011'],
#              ['아시아경제','277'],['이데일리','018'],['조선비즈','366'],['조세일보','123'],['파이낸셜뉴스','014'],['한국경제','015'],
#             ['헤럴드경제','016']]

office_list=pd.DataFrame(office_data,columns=['언론사명','id'])

date_list=pd.read_excel(filename)

In [12]:
def get_news_contents(url,headers):
    url="https://news.naver.com"+url
    driver.get(url)
    time.sleep(3)
    
    text=driver.page_source #브라우저에 보이는 html 문서 그대로의 html을 그대로 가져옴
    
    soup=BeautifulSoup(text,'html.parser')
    
    try:
        title=soup.select_one('#articleTitle').text.strip() #기사 타이틀 

        soup.select_one('#articleBodyContents').script.decompose()
    #     contents=soup.select_one('#articleBodyContents').text.strip()#기사 본문 내용 
    #     contents=clean_text(contents) #contents 내용 정리

        like=soup.select_one('#spiLayer > div._reactionModule.u_likeit > ul > li.u_likeit_list.good > a > span.u_likeit_list_count._count').text
        warm = soup.select_one('#spiLayer > div._reactionModule.u_likeit > ul > li.u_likeit_list.warm > a > span.u_likeit_list_count._count').text
        sad = soup.select_one('#spiLayer > div._reactionModule.u_likeit > ul > li.u_likeit_list.sad > a > span.u_likeit_list_count._count').text
        angry =soup.select_one('#spiLayer > div._reactionModule.u_likeit > ul > li.u_likeit_list.angry > a > span.u_likeit_list_count._count').text

        response=requests.get(url,headers=headers,verify=False)#SSLError 발생으로 verify값은 False로 설정
        root=lxml.html.fromstring(response.content) 
        body=[]
        for text in  root.xpath('//*[@id="articleBodyContents"]/text()'):
            body.append(text)

        contents=' '.join(body)
        contents=clean_text(contents) #contents 내용 정리
        
    except AttributeError:
        title=None; contents=None; like='0'; warm='0'; sad='0'; angry='0'
        
    return title,contents,url,like,warm,sad,angry

In [13]:
def clean_text(tmp):
    punc = '[!"#$%&\'()*+,-./:;<=>?[\]^_`{|}~“”·]'
    reg = re.compile('[a-zA-Z0-9+-_.]+@[a-zA-Z0-9-]+[a-zA-Z0-9-.]+$')
    
    tmp = re.sub(
        '[\{\}\[\]\/?.,;:|\)*~`!^\-_+<>▶▽♡◀━@\#$%&\\\=\(\'\"ⓒ(\n)(\t)]', ' ', tmp)
    tmp = tmp.replace(
        "🇲\u200b🇮\u200b🇱\u200b🇱\u200b🇮\u200b🇪\u200b", "")
    
    tmp = tmp.replace("오류를 우회하기 위한 함수 추가 ", "")
    tmp = tmp.replace("무단 전재 및 재배포 금지", "")
    tmp = tmp.replace("동영상 뉴스 ", "")
    tmp = tmp.replace("앵커", "")
    tmp = tmp.replace('http','')
    tmp = tmp.replace('기자','')
    tmp = tmp.replace('www','')
    tmp = tmp.replace('co','')
    tmp = tmp.replace('kr','')
    tmp = tmp.replace('온라인 제보','')
    
    tmp=re.sub(punc,'',tmp)
    tmp=re.sub(reg,'',tmp)
    
    return tmp

In [14]:
def sttoint(string):
    if re.search(',',string): 
        string=string.replace(',','')
    
    return int(string)

In [15]:
def get_sentiment_index(like,warm,sad,angry):
    label=0
    
    SumPos=like+warm
    SumNeg=sad+angry
    
    SumTotal=SumPos+SumNeg
    Diff=SumPos-SumNeg
    
    try:
        index=(Diff/SumTotal)*100+100
    except ZeroDivisionError: #분모가 0이 되는 상황에서 index는 100으로 설정
        index=100
        
    if index > 100: label=1
    elif index==100: label=0
    else: label=-1
        
    return index,label

In [1]:
#main
df_DataSpec=pd.DataFrame()
df_DataSpec.rename(columns={'언론사 명','전체 뉴스 기사 수','긍정 기사 수','부정 기사 수'})

df_ErrorData=pd.DataFrame()
df_ErrorData.rename(columns={'언론사 명','날짜','url'})

for office_id in office_list['id']:
    office_name=office_list.loc[office_list['id']==office_id,['언론사명']].values[0] #언론사명 
    office_name=office_name[0]
    
    TotalCount=0
    PosCount=0
    NegCount=0
    
    for date in date_list['Date']:
        count=0
        url = "https://news.naver.com/main/ranking/office.nhn?officeId={}&date={}".format(office_id,date)
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36"
        }

        response=requests.get(url,headers=headers,verify=False)
        root=lxml.html.fromstring(response.content)
        url_contents=[]
        df_total=pd.DataFrame()
        df_total.rename(columns={'Date','Title','Contents','URL','Like','Warm','Sad','Angry','Index','Label'})

        for i in range(2,4):
            path='//*[@id="wrap"]/div[4]/div[2]/div['+str(i)+']/ul/li'
            url_contents=[]
            
            if i==2: category="많이 본 뉴스"
            else: category="댓글 많은 뉴스"
                
            for li in root.xpath(path):
                div=li.xpath('div/a')[0]
                
                url_content=div.get('href')
                url_contents.append(url_content)
                
            for url in url_contents:
                
                count+=1
                print(count,date,url)
                title,contents,url,like,warm,sad,angry=get_news_contents(url,headers)
                
                if title is not None: 
                    like=sttoint(like); warm=sttoint(warm); sad=sttoint(sad); angry=sttoint(angry)
                    index,label=get_sentiment_index(like,warm,sad,angry)

                    if label >0 : PosCount+=1
                    else: NegCount+=1

                    df_tmp=pd.DataFrame({'Category':[category],'Date':[date],'Title':[title],'Contents':[contents],'URL':[url],
                                                'Like':[like],'Warm':[warm],'Sad':[sad],'Angry':[angry],'Index':[index],'Label':[label]})

                    df_total=df_total.append(df_tmp,ignore_index=True)
                    TotalCount+=1
                else: 
                    df_tmp=pd.DataFrame({'언론사 명':[office_name],'date':[date],'url':[url]})
                    df_ErrorData=df_ErrorData.append(df_tmp,ignore_index=True)
                    print('error')
                    
        df_total.to_excel('file_name'+'.xlsx')
    
    df_tmp=pd.DataFrame({'언론사 명':[office_name],'전체 뉴스 기사 수':[TotalCount],'긍정 기사 수':[PosCount],'부정 기사 수':[NegCount]})
    df_DataSpec=df_DataSpec.append(df_tmp,ignore_index=True)
    